# Tutorial 1: Principal Component Analysis (PCA)

**CSCI 8945: Advanced Representation Learning**  
**Fall 2025**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/yourusername/csci8945-advanced-representation-learning/blob/main/notebooks/tutorial1_pca.ipynb)

---

## Learning Objectives

By the end of this tutorial, you will be able to:

1. **Understand the mathematical foundations** of Principal Component Analysis (PCA)
2. **Implement PCA from scratch** using NumPy
3. **Apply PCA** to real datasets for dimensionality reduction
4. **Interpret PCA results** and visualize principal components
5. **Compare custom implementation** with sklearn's PCA
6. **Understand the limitations** and assumptions of PCA

## Table of Contents

1. [Introduction to Dimensionality Reduction](#1-introduction)
2. [Mathematical Foundations of PCA](#2-mathematical-foundations)
3. [Implementation from Scratch](#3-implementation-from-scratch)
4. [Application to Real Data](#4-application-to-real-data)
5. [Comparison with sklearn](#5-comparison-with-sklearn)
6. [Visualization and Interpretation](#6-visualization-and-interpretation)
7. [Limitations and Extensions](#7-limitations-and-extensions)
8. [Exercises](#8-exercises)

---

## 1. Introduction to Dimensionality Reduction {#1-introduction}

**Dimensionality reduction** is a fundamental problem in machine learning and data science. Real-world datasets often contain hundreds or thousands of features, but not all of these features are equally important for understanding the underlying structure of the data.

### Why do we need dimensionality reduction?

1. **Curse of dimensionality**: As the number of dimensions increases, data becomes sparse
2. **Computational efficiency**: Fewer dimensions mean faster algorithms
3. **Visualization**: Humans can only visualize 2D or 3D data effectively
4. **Noise reduction**: Remove irrelevant features that might be noise
5. **Storage**: Reduce memory and storage requirements

### Principal Component Analysis (PCA)

PCA is one of the most widely used techniques for dimensionality reduction. It finds the directions (principal components) along which the data varies the most and projects the data onto these directions.

**Key Idea**: Find a lower-dimensional representation that preserves as much variance as possible.


## 2. Mathematical Foundations of PCA {#2-mathematical-foundations}

### 2.1 Problem Setup

Let's say we have a dataset $\mathbf{X} \in \mathbb{R}^{n \times d}$ where:
- $n$ is the number of samples
- $d$ is the number of features/dimensions
- Each row $\mathbf{x}_i$ represents one data sample

**Goal**: Find a lower-dimensional representation $\mathbf{Z} \in \mathbb{R}^{n \times k}$ where $k < d$.

### 2.2 Mathematical Formulation

#### Step 1: Center the data
First, we center the data by subtracting the mean:
$$\tilde{\mathbf{X}} = \mathbf{X} - \boldsymbol{\mu}$$
where $\boldsymbol{\mu} = \frac{1}{n}\sum_{i=1}^{n} \mathbf{x}_i$ is the sample mean.

#### Step 2: Compute the covariance matrix
The sample covariance matrix is:
$$\mathbf{C} = \frac{1}{n-1}\tilde{\mathbf{X}}^T\tilde{\mathbf{X}}$$

#### Step 3: Eigenvalue decomposition
Find the eigenvalues $\lambda_i$ and eigenvectors $\mathbf{v}_i$ of the covariance matrix:
$$\mathbf{C}\mathbf{v}_i = \lambda_i \mathbf{v}_i$$

The eigenvectors are the **principal components** and the eigenvalues represent the **variance** explained by each component.

#### Step 4: Select top-k components
Sort eigenvalues in descending order: $\lambda_1 \geq \lambda_2 \geq \ldots \geq \lambda_d$

Select the first $k$ eigenvectors to form the projection matrix:
$$\mathbf{W} = [\mathbf{v}_1, \mathbf{v}_2, \ldots, \mathbf{v}_k] \in \mathbb{R}^{d \times k}$$

#### Step 5: Project the data
The lower-dimensional representation is:
$$\mathbf{Z} = \tilde{\mathbf{X}}\mathbf{W}$$

### 2.3 Key Properties

1. **Variance maximization**: The first principal component maximizes the variance of the projected data
2. **Orthogonality**: Principal components are orthogonal to each other
3. **Reconstruction**: Original data can be approximately reconstructed as $\hat{\mathbf{X}} = \mathbf{Z}\mathbf{W}^T + \boldsymbol{\mu}$
4. **Variance explained**: The ratio $\frac{\lambda_i}{\sum_j \lambda_j}$ gives the proportion of variance explained by the $i$-th component


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris, load_digits, make_blobs
from sklearn.decomposition import PCA as SklearnPCA
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Configure matplotlib
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")


## 3. Implementation from Scratch {#3-implementation-from-scratch}

Let's implement PCA from scratch following the mathematical steps we outlined above. This will help us understand exactly what's happening under the hood.


In [ ]:
class PCA:
    """
    Principal Component Analysis implemented from scratch.
    
    Parameters:
    -----------
    n_components : int, optional (default=None)
        Number of components to keep. If None, keep all components.
    """
    
    def __init__(self, n_components=None):
        self.n_components = n_components
        self.components_ = None
        self.explained_variance_ = None
        self.explained_variance_ratio_ = None
        self.mean_ = None
        
    def fit(self, X):
        """
        Fit PCA on the data X.
        
        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
            Training data
        """
        # Convert to numpy array if needed
        X = np.array(X)
        n_samples, n_features = X.shape
        
        # Step 1: Center the data
        self.mean_ = np.mean(X, axis=0)
        X_centered = X - self.mean_
        
        # Step 2: Compute covariance matrix
        # Note: Using (n-1) for unbiased estimator
        cov_matrix = np.cov(X_centered.T)
        
        # Step 3: Eigenvalue decomposition
        eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)
        
        # Step 4: Sort eigenvalues and eigenvectors in descending order
        idx = np.argsort(eigenvalues)[::-1]
        eigenvalues = eigenvalues[idx]
        eigenvectors = eigenvectors[:, idx]
        
        # Step 5: Select number of components
        if self.n_components is None:
            self.n_components = n_features
        
        self.components_ = eigenvectors[:, :self.n_components].T
        self.explained_variance_ = eigenvalues[:self.n_components]
        self.explained_variance_ratio_ = self.explained_variance_ / np.sum(eigenvalues)
        
        return self
    
    def transform(self, X):
        """
        Apply dimensionality reduction to X.
        
        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
            Input data
            
        Returns:
        --------
        X_transformed : array, shape (n_samples, n_components)
            Transformed data
        """
        X = np.array(X)
        X_centered = X - self.mean_
        return np.dot(X_centered, self.components_.T)
    
    def fit_transform(self, X):
        """
        Fit PCA and transform the data.
        """
        self.fit(X)
        return self.transform(X)
    
    def inverse_transform(self, X_transformed):
        """
        Transform data back to original space.
        
        Parameters:
        -----------
        X_transformed : array-like, shape (n_samples, n_components)
            Transformed data
            
        Returns:
        --------
        X_reconstructed : array, shape (n_samples, n_features)
            Reconstructed data
        """
        return np.dot(X_transformed, self.components_) + self.mean_
    
    def get_cumulative_variance_ratio(self):
        """
        Get cumulative explained variance ratio.
        """
        return np.cumsum(self.explained_variance_ratio_)

# Test our implementation with a simple example
print("PCA class implemented successfully!")

# Create a simple 2D dataset for testing
np.random.seed(42)
X_simple = np.random.randn(100, 2)
X_simple[:, 1] = X_simple[:, 0] + 0.5 * np.random.randn(100)  # Create correlation

# Fit PCA
pca_custom = PCA(n_components=2)
X_transformed = pca_custom.fit_transform(X_simple)

print(f"Original data shape: {X_simple.shape}")
print(f"Transformed data shape: {X_transformed.shape}")
print(f"Explained variance ratio: {pca_custom.explained_variance_ratio_}")
print(f"Cumulative explained variance: {pca_custom.get_cumulative_variance_ratio()}")


## 4. Application to Real Data {#4-application-to-real-data}

Let's apply our PCA implementation to the famous Iris dataset and visualize the results.


In [ ]:
# Load the Iris dataset
iris = load_iris()
X_iris = iris.data  # 4 features: sepal length, sepal width, petal length, petal width
y_iris = iris.target
feature_names = iris.feature_names
target_names = iris.target_names

print("Iris Dataset Information:")
print(f"Shape: {X_iris.shape}")
print(f"Features: {feature_names}")
print(f"Classes: {target_names}")
print(f"Class distribution: {np.bincount(y_iris)}")

# Standardize the data (important for PCA)
scaler = StandardScaler()
X_iris_scaled = scaler.fit_transform(X_iris)

print(f"\nOriginal data statistics:")
print(f"Mean: {np.mean(X_iris, axis=0)}")
print(f"Std:  {np.std(X_iris, axis=0)}")

print(f"\nScaled data statistics:")
print(f"Mean: {np.mean(X_iris_scaled, axis=0)}")
print(f"Std:  {np.std(X_iris_scaled, axis=0)}")

# Apply our PCA implementation
pca_iris = PCA(n_components=2)
X_iris_pca = pca_iris.fit_transform(X_iris_scaled)

print(f"\nPCA Results:")
print(f"Explained variance ratio: {pca_iris.explained_variance_ratio_}")
print(f"Cumulative explained variance: {pca_iris.get_cumulative_variance_ratio()}")
print(f"Total variance explained by 2 components: {np.sum(pca_iris.explained_variance_ratio_):.3f}")

# Visualize the results
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Original data (first two features)
scatter1 = axes[0].scatter(X_iris_scaled[:, 0], X_iris_scaled[:, 1], 
                          c=y_iris, cmap='viridis', alpha=0.7)
axes[0].set_xlabel(feature_names[0])
axes[0].set_ylabel(feature_names[1])
axes[0].set_title('Original Data (First 2 Features)')
axes[0].grid(True, alpha=0.3)

# PCA transformed data
scatter2 = axes[1].scatter(X_iris_pca[:, 0], X_iris_pca[:, 1], 
                          c=y_iris, cmap='viridis', alpha=0.7)
axes[1].set_xlabel('First Principal Component')
axes[1].set_ylabel('Second Principal Component')
axes[1].set_title('PCA Transformed Data')
axes[1].grid(True, alpha=0.3)

# Add colorbar
plt.colorbar(scatter2, ax=axes[1])

plt.tight_layout()
plt.show()

# Show principal components
print(f"\nPrincipal Components:")
for i, component in enumerate(pca_iris.components_):
    print(f"PC{i+1}: {component}")
    print(f"      Features contribution: {dict(zip(feature_names, component))}")
    print()


## 5. Comparison with sklearn {#5-comparison-with-sklearn}

Let's compare our implementation with sklearn's PCA to verify our results are correct.


In [ ]:
# Apply sklearn's PCA
pca_sklearn = SklearnPCA(n_components=2)
X_iris_sklearn = pca_sklearn.fit_transform(X_iris_scaled)

# Compare results
print("Comparison between our PCA and sklearn's PCA:")
print("=" * 50)

print(f"\nExplained Variance Ratio:")
print(f"Our PCA:     {pca_iris.explained_variance_ratio_}")
print(f"Sklearn PCA: {pca_sklearn.explained_variance_ratio_}")
print(f"Difference:  {np.abs(pca_iris.explained_variance_ratio_ - pca_sklearn.explained_variance_ratio_)}")

print(f"\nExplained Variance:")
print(f"Our PCA:     {pca_iris.explained_variance_}")
print(f"Sklearn PCA: {pca_sklearn.explained_variance_}")
print(f"Difference:  {np.abs(pca_iris.explained_variance_ - pca_sklearn.explained_variance_)}")

print(f"\nPrincipal Components (absolute values):")
print(f"Our PCA PC1:     {np.abs(pca_iris.components_[0])}")
print(f"Sklearn PC1:     {np.abs(pca_sklearn.components_[0])}")
print(f"Difference PC1:  {np.abs(np.abs(pca_iris.components_[0]) - np.abs(pca_sklearn.components_[0]))}")

print(f"\nOur PCA PC2:     {np.abs(pca_iris.components_[1])}")
print(f"Sklearn PC2:     {np.abs(pca_sklearn.components_[1])}")
print(f"Difference PC2:  {np.abs(np.abs(pca_iris.components_[1]) - np.abs(pca_sklearn.components_[1]))}")

# Note: Signs of eigenvectors can be flipped, so we compare absolute values
# The direction doesn't matter for PCA, only the subspace spanned by the eigenvectors

# Compare transformed data (accounting for potential sign flips)
correlation_pc1 = np.corrcoef(X_iris_pca[:, 0], X_iris_sklearn[:, 0])[0, 1]
correlation_pc2 = np.corrcoef(X_iris_pca[:, 1], X_iris_sklearn[:, 1])[0, 1]

print(f"\nCorrelation between transformed data:")
print(f"PC1 correlation: {np.abs(correlation_pc1):.6f}")
print(f"PC2 correlation: {np.abs(correlation_pc2):.6f}")

# Visualize comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Our PCA
scatter1 = axes[0].scatter(X_iris_pca[:, 0], X_iris_pca[:, 1], 
                          c=y_iris, cmap='viridis', alpha=0.7)
axes[0].set_xlabel('First Principal Component')
axes[0].set_ylabel('Second Principal Component')
axes[0].set_title('Our PCA Implementation')
axes[0].grid(True, alpha=0.3)

# Sklearn PCA
scatter2 = axes[1].scatter(X_iris_sklearn[:, 0], X_iris_sklearn[:, 1], 
                          c=y_iris, cmap='viridis', alpha=0.7)
axes[1].set_xlabel('First Principal Component')
axes[1].set_ylabel('Second Principal Component')
axes[1].set_title('Sklearn PCA')
axes[1].grid(True, alpha=0.3)

# Difference
scatter3 = axes[2].scatter(X_iris_pca[:, 0] - X_iris_sklearn[:, 0], 
                          X_iris_pca[:, 1] - X_iris_sklearn[:, 1], 
                          c=y_iris, cmap='viridis', alpha=0.7)
axes[2].set_xlabel('PC1 Difference')
axes[2].set_ylabel('PC2 Difference')
axes[2].set_title('Difference (Our - Sklearn)')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Reconstruction test
X_reconstructed_ours = pca_iris.inverse_transform(X_iris_pca)
X_reconstructed_sklearn = pca_sklearn.inverse_transform(X_iris_sklearn)

reconstruction_error_ours = np.mean((X_iris_scaled - X_reconstructed_ours) ** 2)
reconstruction_error_sklearn = np.mean((X_iris_scaled - X_reconstructed_sklearn) ** 2)

print(f"\nReconstruction Error (MSE):")
print(f"Our PCA:     {reconstruction_error_ours:.8f}")
print(f"Sklearn PCA: {reconstruction_error_sklearn:.8f}")
print(f"Difference:  {np.abs(reconstruction_error_ours - reconstruction_error_sklearn):.8f}")

print("\n✅ Our implementation matches sklearn's PCA!")


## 6. Visualization and Interpretation {#6-visualization-and-interpretation}

Let's explore different ways to visualize and interpret PCA results.


In [ ]:
# Comprehensive visualization of PCA results

# 1. Scree Plot - showing explained variance by each component
pca_all = PCA()  # Fit PCA with all components
pca_all.fit(X_iris_scaled)

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Scree plot
axes[0, 0].plot(range(1, len(pca_all.explained_variance_ratio_) + 1), 
                pca_all.explained_variance_ratio_, 'bo-', linewidth=2, markersize=8)
axes[0, 0].set_xlabel('Principal Component')
axes[0, 0].set_ylabel('Explained Variance Ratio')
axes[0, 0].set_title('Scree Plot')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].set_xticks(range(1, len(pca_all.explained_variance_ratio_) + 1))

# Cumulative explained variance
cumulative_var = pca_all.get_cumulative_variance_ratio()
axes[0, 1].plot(range(1, len(cumulative_var) + 1), cumulative_var, 'ro-', linewidth=2, markersize=8)
axes[0, 1].axhline(y=0.95, color='k', linestyle='--', alpha=0.7, label='95% threshold')
axes[0, 1].axhline(y=0.90, color='gray', linestyle='--', alpha=0.7, label='90% threshold')
axes[0, 1].set_xlabel('Number of Components')
axes[0, 1].set_ylabel('Cumulative Explained Variance')
axes[0, 1].set_title('Cumulative Explained Variance')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].legend()
axes[0, 1].set_xticks(range(1, len(cumulative_var) + 1))

# Component loadings (biplot style)
feature_names_short = ['SL', 'SW', 'PL', 'PW']  # Shortened names
colors = ['red', 'blue', 'green', 'orange']

for i, (feature, color) in enumerate(zip(feature_names_short, colors)):
    axes[1, 0].arrow(0, 0, pca_iris.components_[0, i]*3, pca_iris.components_[1, i]*3,
                     head_width=0.1, head_length=0.1, fc=color, ec=color, linewidth=2, label=feature)
    axes[1, 0].text(pca_iris.components_[0, i]*3.2, pca_iris.components_[1, i]*3.2, 
                    feature, fontsize=10, color=color, weight='bold')

# Add data points
scatter = axes[1, 0].scatter(X_iris_pca[:, 0], X_iris_pca[:, 1], 
                            c=y_iris, cmap='viridis', alpha=0.6, s=50)
axes[1, 0].set_xlabel('First Principal Component')
axes[1, 0].set_ylabel('Second Principal Component')
axes[1, 0].set_title('PCA Biplot')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].legend()

# Component contributions heatmap
component_matrix = np.abs(pca_iris.components_)
im = axes[1, 1].imshow(component_matrix, cmap='Blues', aspect='auto')
axes[1, 1].set_xticks(range(len(feature_names)))
axes[1, 1].set_xticklabels(feature_names_short, rotation=45)
axes[1, 1].set_yticks(range(len(pca_iris.components_)))
axes[1, 1].set_yticklabels([f'PC{i+1}' for i in range(len(pca_iris.components_))])
axes[1, 1].set_title('Component Loadings Heatmap')

# Add text annotations
for i in range(component_matrix.shape[0]):
    for j in range(component_matrix.shape[1]):
        text = axes[1, 1].text(j, i, f'{component_matrix[i, j]:.2f}',
                              ha="center", va="center", color="black", fontweight='bold')

plt.colorbar(im, ax=axes[1, 1])
plt.tight_layout()
plt.show()

# Print interpretation
print("Interpretation of PCA Results:")
print("=" * 40)
print(f"• PC1 explains {pca_iris.explained_variance_ratio_[0]:.1%} of the variance")
print(f"• PC2 explains {pca_iris.explained_variance_ratio_[1]:.1%} of the variance")
print(f"• Together, they explain {np.sum(pca_iris.explained_variance_ratio_):.1%} of the total variance")

print(f"\nPC1 is most influenced by:")
pc1_influences = [(feature_names[i], abs(pca_iris.components_[0, i])) for i in range(len(feature_names))]
pc1_influences.sort(key=lambda x: x[1], reverse=True)
for feature, influence in pc1_influences:
    print(f"  • {feature}: {influence:.3f}")

print(f"\nPC2 is most influenced by:")
pc2_influences = [(feature_names[i], abs(pca_iris.components_[1, i])) for i in range(len(feature_names))]
pc2_influences.sort(key=lambda x: x[1], reverse=True)
for feature, influence in pc2_influences:
    print(f"  • {feature}: {influence:.3f}")

# Show how many components needed for different variance thresholds
for threshold in [0.8, 0.9, 0.95, 0.99]:
    n_components_needed = np.argmax(cumulative_var >= threshold) + 1
    print(f"\nTo explain {threshold:.0%} of variance, you need {n_components_needed} component(s)")
    print(f"Actual variance explained: {cumulative_var[n_components_needed-1]:.1%}")


## 7. Limitations and Extensions {#7-limitations-and-extensions}

### 7.1 Limitations of PCA

1. **Linear assumption**: PCA only captures linear relationships
2. **Interpretation**: Principal components may not have clear physical meaning
3. **Scaling sensitivity**: Results depend heavily on feature scaling
4. **Outlier sensitivity**: PCA is sensitive to outliers
5. **Global technique**: Uses all data points to find components

### 7.2 When PCA works well

- Features are correlated
- Linear relationships dominate
- Data is roughly Gaussian
- You need dimensionality reduction
- Interpretability is not critical

### 7.3 When PCA might not work well

- Strong non-linear relationships
- Categorical or binary features
- Very sparse data
- When you need interpretable features

### 7.4 Extensions and Alternatives

1. **Kernel PCA**: Handles non-linear relationships
2. **Sparse PCA**: For interpretable components
3. **Robust PCA**: Less sensitive to outliers
4. **Incremental PCA**: For large datasets
5. **t-SNE**: For visualization of non-linear structures
6. **UMAP**: Another non-linear dimensionality reduction technique


## 8. Exercises {#8-exercises}

Now it's time to practice! Try these exercises to solidify your understanding of PCA.

### Exercise 1: PCA on Digits Dataset (Easy)
Load the digits dataset from sklearn and apply PCA to reduce it from 64 dimensions to 2. Visualize the results and calculate how much variance is explained.

### Exercise 2: Effect of Scaling (Medium)
Compare PCA results on the Iris dataset with and without standardization. What differences do you observe? Why does this happen?

### Exercise 3: Choosing Number of Components (Medium)
Using the digits dataset, create a plot showing reconstruction error vs. number of components. Find the "elbow point" where adding more components doesn't significantly reduce the error.

### Exercise 4: PCA for Image Compression (Hard)
Implement a simple image compression algorithm using PCA. Load a grayscale image, apply PCA, and reconstruct the image using different numbers of components. Show the trade-off between compression ratio and image quality.

### Exercise 5: Custom Dataset (Hard)
Create a synthetic 3D dataset where the data lies approximately on a 2D plane. Add some noise and apply PCA. Visualize the original data and the principal components. Does PCA correctly identify the underlying 2D structure?


## Summary

Congratulations! You've successfully completed the PCA tutorial. Here's what you've learned:

✅ **Mathematical foundations** of PCA  
✅ **Implementation from scratch** using NumPy  
✅ **Application** to real datasets  
✅ **Comparison** with sklearn implementation  
✅ **Visualization and interpretation** techniques  
✅ **Limitations** and when to use PCA

### Key Takeaways

1. **PCA finds directions of maximum variance** in the data
2. **Principal components are orthogonal** to each other
3. **Standardization is crucial** for meaningful results
4. **PCA is a linear technique** - it won't capture non-linear relationships
5. **Dimensionality reduction** involves a trade-off between simplicity and information loss

### Next Steps

In the next tutorial, we'll explore **Kernel PCA** and other non-linear dimensionality reduction techniques that can capture more complex patterns in data.

---

**Happy learning! 🎓**
